In [1]:
from osgeo import gdal, osr
from osgeo import osr
import geopandas as gpd
from osgeo import ogr
from shapely.geometry import Polygon, Point
from geopandas.geoseries import *
import re

In [2]:
import tarfile
import pandas as pd
import datetime as dt
import os
import glob
import subprocess
import os.path
from os import path
import json
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
from terrautils.spatial import scanalyzer_to_latlon

In [3]:
# Grabs the thermal raw data tar file for the Gantry's metadata for the images taken at the specified date
class JSON:
    def get_tar(season, date):
        command = f'iget -rKTPf -N 0 /iplant/home/shared/terraref/ua-mac/raw_tars/{season}/flirIrCamera/flirIrCamera-{date}.tar'
        subprocess.call(command, shell = True)
        command = f'tar -xvf flirIrCamera-{date}.tar'
        subprocess.call(command, shell = True)

# Finds the individual json files and adds them to the filepath (end up with a list of paths for each individual json file)
    def pathlist(season, date):
        json_data = JSON.get_tar(season, date)
        pathlist = Path(f"./flirIrCamera/{date}/").glob('**/*.json')
        JSON_path_list = []
        for path in pathlist:
            path_str = str(path)
            JSON_path_list.append(path_str)
        return JSON_path_list

# Uses the pathlists and searches each one
# Gathers the time, image name, gantry_x and gantry_y information from the metadata of each individual image and adds to dictionary
    def time_dict(season, date):
        file_path_list = JSON.pathlist(season, date)
        JSON_dict = dict()
        for file in file_path_list:
            path_metadata = glob.glob(f'{file}')
            metadata = str(path_metadata)[2:-2]
            with open(metadata) as f:
                meta = json.load(f)['lemnatec_measurement_metadata']
                time = (meta['gantry_system_variable_metadata']['time'])
                gantry_x = float(meta['gantry_system_variable_metadata']['position x [m]'])
                gantry_y = float(meta['gantry_system_variable_metadata']['position y [m]'])
                filename = os.path.basename(metadata)
            if JSON is not JSON_dict:
                JSON_dict[time, filename, gantry_x, gantry_y] = "Date, Time, Gantry_x, Gantry_y"
            else:
                print("JSON already in Dictionary")
        return sorted(JSON_dict)

# Searches through the dictionary created and creates a dataframe of the information in the dictionary
    def time_df(season, date):
        JSON_time_d = JSON.time_dict(season, date)
        JSON_time_df = pd.DataFrame.from_dict(JSON_time_d)
        JSON_time_df.columns = ['Date and Time', 'Image Name', 'Gantry_x', 'Gantry_y']
        return JSON_time_df

# Converts the gantry coordinates into GPS coordinates (using 'terrautils')
    def GPS_coord (season, date):
        data = JSON.time_df(season, date)
        gantry_x_pos = data['Gantry_x']
        gantry_y_pos = data['Gantry_y']
        GPS_latlon = scanalyzer_to_latlon(gantry_x_pos, gantry_y_pos)
        GPS_df = pd.DataFrame(GPS_latlon)
        GPS_latlon_df = GPS_df.transpose()
        GPS_latlon_df.columns = ['GPS_lat', 'GPS_lon']
        data['GPS_lat'] = GPS_latlon_df['GPS_lat']
        data['GPS_lon'] = GPS_latlon_df['GPS_lon']
        return data

In [4]:
image_gps = JSON.GPS_coord('season_10_yr_2020', '2020-03-03')

In [5]:
lat,lon = image_gps.loc[1]['GPS_lat'], image_gps.loc[1]['GPS_lon']

In [6]:
shp = gpd.read_file('/Users/sebastiancalleja/Desktop/season10_multi_latlon_geno.geojson')

In [7]:
point_list = []
poly_list = []

class Get_plot:
    def to_point (point_data, geom_data):
        for i, row in point_data.iterrows():
            lat = point_data['GPS_lat'].loc[i]
            lon = point_data['GPS_lon'].loc[i]
            points = Point(lon,lat)
            point_list.append(points)
            point_list_df = gpd.GeoDataFrame(point_list)
        return point_list_df

    def poly (point_data, geom_data):
        for i, row in geom_data.iterrows():
            poly_data = geom_data['geometry'].loc[i]
            poly_list.append(poly_data)
            poly_list_df = gpd.GeoDataFrame(poly_list)
        return poly_list_df

In [8]:
image_gps['Points'] = Get_plot.to_point(image_gps, shp)

In [9]:
#GPS_points.set_index(0)
#GPS_points.rename(columns={0: 'Points'}, inplace=True)

In [10]:
#data_all = gpd.GeoDataFrame(shp['geometry'])

In [11]:
#GPS_points.at[i,"plot"]

In [12]:
def intersection(point):
    within = point.within
    plot = None
    intersection_list = []
    for i, row in shp.iterrows():
        polygon = row['geometry']
        intersection = within(polygon)
        if intersection == True:
            plot = [row['ID']]
            intersection_list.append(plot)
    return intersection_list

In [13]:
image_gps["plot"] = None
for i, row in image_gps.iterrows():
    point = row['Points']
    plot = intersection(point)
    image_gps.at[i,'plot'] = plot

In [17]:
# Find points that had no intersecrtion with plots
# We want to create a polygon out of the image and see which plots they intersect (rather than use the center point)

In [15]:
no_intersection = image_gps.loc[(image_gps['plot'].str.len() == 0),:]

In [16]:
no_intersection

,Date and Time,Image Name,Gantry_x,Gantry_y,GPS_lat,GPS_lon,Points,plot
1,03/03/2020 08:45:35,4cb25050-63bf-4669-831f-2c6c7a5c31fa_metadata....,207.938000,0.522,33.076434,-111.974816,POINT (-111.974816085573 33.07643428493439),[]
3,03/03/2020 08:45:38,d71b8eef-4830-4839-94cb-a5d3e036684f_metadata....,207.938000,1.515,33.076434,-111.974827,POINT (-111.9748267089786 33.07643427173947),[]
5,03/03/2020 08:45:41,693946d9-8d36-4d16-a58f-c2c506dd84ec_metadata....,207.938000,2.529,33.076434,-111.974838,POINT (-111.9748375570483 33.07643425826458),[]
7,03/03/2020 08:45:44,166f5b50-ef63-45c5-95e4-5377892e7e49_metadata....,207.938000,3.517,33.076434,-111.974848,POINT (-111.9748481269624 33.07643424513428),[]
9,03/03/2020 08:45:47,5ca936c1-0c42-4c16-9abf-c3e7f2bfeada_metadata....,207.938000,4.518,33.076434,-111.974859,POINT (-111.9748588359543 33.07643423183031),[]
...,...,...,...,...,...,...,...,...
9260,03/03/2020 13:27:39,7b33391c-5188-4db8-bf05-496bee9c714e_metadata....,3.800492,4.509,33.074593,-111.974858,POINT (-111.9748581091793 33.07459260203276),[]
9262,03/03/2020 13:27:42,53793107-5f8b-4205-bc55-f38ef2e7664f_metadata....,3.800988,3.501,33.074593,-111.974847,POINT (-111.9748473255256 33.07459261990022),[]
9264,03/03/2020 13:27:45,bfb39761-e5f6-4156-839e-8fa2bec1dde7_metadata....,3.800492,2.499,33.074593,-111.974837,POINT (-111.9748366060572 33.07459262873769),[]
9266,03/03/2020 13:27:48,2141e0d5-7395-48d3-923b-eebe74586e1a_metadata....,3.800988,1.509,33.074593,-111.974826,POINT (-111.9748260149688 33.07459264636417),[]
